# Use Case 2: find the "*shadows*"/"*holes*" in the access to nature (*green spaces, parks, ...*)

## First test: accessibility from multiple parks

We will use what we have done in the previous Use Case and a little bit of complexity:
1. find parks as a polygons
2. make a buffer around each park & get network nodes in these buffers
3. measure accessibility from these nodes
4. Make a map of *shadows*

### 1. Find parks' polygons

To get parks data, we are going to use this tools/libraries: 
* [Bounding box for overpass](https://wiki.openstreetmap.org/wiki/Overpass_API#The_map_query)
* [Overpass turbo](http://overpass-turbo.eu/#)
* [Overpass API Python](https://github.com/mvexel/overpass-api-python-wrapper)

#### Get data from OSM

In [ ]:
from overpass_methods import get_OSM_poly

#bbox (SOUTH, WEST, NORTH, EAST)
bbox = (45.7,4.82,45.8,4.99)
key = "leisure"
value = "park"

features = get_OSM_poly(bbox, key, value)

#### Map parks OSM data

In [ ]:
from bokeh.io import show, output_file, output_notebook
from bokeh.models import GeoJSONDataSource, ColorBar, LogColorMapper, LogTicker, ColumnDataSource
from bokeh.plotting import figure
from bokeh.tile_providers import get_provider, Vendors
from bokeh.io import export_png
from bokeh.layouts import gridplot
from bokeh.palettes import Viridis6
from bokeh.transform import linear_cmap
import geopandas as gpd
import json

output_notebook()

#Prepare parks GeoJSON data
parks = gpd.GeoDataFrame.from_features(features['features'])
parks.crs = {"init":"epsg:4326"}
parks = parks.to_crs({"init":"epsg:3857"}) #change projection for visualisation purposes
parks.to_file("./data/parks.geojson", driver="GeoJSON")

with open("./data/parks.geojson") as f:
    d = json.load(f)
    parks_source = GeoJSONDataSource(
        geojson=json.dumps(d)
    )

# Create Bokeh figures
p1 = figure(
        width = 800,
        height = 600,
        output_backend="webgl"
    )

# Add background tiles
p1.add_tile(get_provider(Vendors.STAMEN_TONER_BACKGROUND), alpha=0.2)

# # Add parks
p1.patches(
    xs="xs",
    ys="ys",
    fill_color="green",
    fill_alpha=0.7,
    line_color="white",
    line_alpha=0.5,
    line_width=1,
    source=parks_source,
    legend="Parks"
)

p1.axis.visible = False

# Export to png for illustration purpose in markdown
export_png(p1, filename="./img/parks.png")


In [ ]:
show(p1)

<img src="./img/parks.png" width="70%">

#### Get graph

> ***Do not use this part of code if you already have ```nodes JSON``` and ```edges JSON``` files***

In [ ]:
import osmnx as ox

from graph_utils import graph_to_df, graph_with_time

# Parameters
bbox = [4.73,45.66,5.05,45.86] #from Geofabrik Tile Calculator, [left, bottom, right, top]

# Get graph from OSM data
graph = ox.graph_from_bbox(
    bbox[3],
    bbox[1],
    bbox[2],
    bbox[0],
    network_type="walk"
) #bbox => north, south, east, west

# Add time as weight in graph (based on walk speed)
walk_distance = 5000 #in meters for 1 hour trip
graph = graph_with_time(graph, walk_distance)

#Write graph to disk as 2 files (nodes and edges)
nodes_path = "./data/graph/nodes.json"
edges_path = "./data/graph/edges.json"
graph_to_df(graph, edges_path, nodes_path)

#### Load graph

In [ ]:
from graph_utils import df_to_graph

# Import graph from json files and transform to NetworkX MultiDiGraph
nodes_path = "./data/graph/nodes.json"
edges_path = "./data/graph/edges.json"
G = df_to_graph(edges_path, nodes_path)

#### Get center nodes

In [ ]:
import geopandas as gpd

from isochrone import Accessibility
from get_nearest_points import GetCenterNodes
from graph_utils import graph_to_gdf_points 
from get_holes import get_all_center_nodes

#Parameters
epsg_polys = 3857
epsg_graph = 4326
epsg_metric = 2154
lat = "x"
lng = "y"

# Get Points GeoDataframe from graph
points = graph_to_gdf_points(G, lat, lng, epsg_graph)
# Reproject points to metric
points = points.to_crs(
    {
        "init":"epsg:{}".format(epsg_metric)
    }
)

# Reproject polys to metric
parks = gpd.GeoDataFrame.from_file("./data/parks.geojson")
parks.crs = {
    "init":"epsg:{}".format(epsg_polys)
}
parks = parks.to_crs(
    {
        "init":"epsg:{}".format(epsg_metric)
    }
)

#Get polys as list
park_polys = parks["geometry"].to_list()
poly_names = parks["name"].to_list()
        
center_nodes = get_all_center_nodes(park_polys, points, lat, lng, poly_names=poly_names)

In [ ]:
import time
import datetime

start = time.time()

tmp = []
for center in center_nodes:
    tmp.extend(center)

end  = time.time()

print (str(datetime.timedelta(seconds=end-start)))

### 2. Make a buffer & get network nodes in this buffer

We use our own Python class and methods to get the nearest nodes from the park polygon. 

In [ ]:
from isochrone import Accessibility

#Delete duplicates
set_center_nodes = list(set(tmp))

# Parameters
trip_times = [5,10,15] #in minutes
distance_buffer = 40 #in meters

access = Accessibility(
    G, 
    trip_times, 
    distance_buffer,
    center_nodes=set_center_nodes,
)

access.get_results()

#### Simple plot

In [ ]:
access.lines.origin.plot(column='duration', cmap='viridis');

#### Visualisation with Bokeh

In [ ]:
from bokeh.io import show, output_file, output_notebook
from bokeh.models import GeoJSONDataSource, ColorBar, LogColorMapper, LogTicker, ColumnDataSource
from bokeh.plotting import figure
from bokeh.tile_providers import get_provider, Vendors
from bokeh.io import export_png
from bokeh.layouts import gridplot
from bokeh.palettes import Viridis6, Viridis3
from bokeh.transform import linear_cmap
import geopandas as gpd
import json

from bokeh_transform import geojson_to_datasource

output_notebook()

union_name = "./data/Lyon_metropole/union_park.geojson"
park_name = "./data/parks.geojson"

# Save GeoDataframe as GeoJSON
access.union.vis.to_file(union_name, driver="GeoJSON")

# Open GeoJSON as GeoDataSources for Bokeh webmapping
with open(union_name) as f:
    d = json.load(f)
    union_source = GeoJSONDataSource(
        geojson=json.dumps(d)
    )
    
with open(park_name) as f:
    d = json.load(f)
    park_source = GeoJSONDataSource(
        geojson=json.dumps(d)
    )
    
# Create color mapper
mapper = linear_cmap(
    field_name='duration', 
    palette=Viridis3, 
    low=0.01, 
    high=15
)
    
# Create Bokeh figures
p1 = figure(
        width = 800,
        height = 600,
        output_backend="webgl"
    )

# Add background tiles
p1.add_tile(get_provider(Vendors.STAMEN_TONER_BACKGROUND), alpha=0.2)

# Transform union_source to Bokeh multi_polygon format
union_name = "./data/Lyon_metropole/union_park.geojson"
multi_polys_source = geojson_to_datasource(union_name)

# Add union of buffered lines
p1.multi_polygons(
    xs="xs",
    ys="ys",
    fill_color="color",
    fill_alpha=0.7,
    line_color="white",
    line_alpha=0.0,
    line_width=1,
    source=multi_polys_source,
    legend="Union"
)

p1.patches(
    xs="xs",
    ys="ys",
    fill_color="red",
    fill_alpha=0.5,
    line_color="white",
    line_alpha=1.0,
    line_width=1,
    source=park_source,
    legend="Parks"
)

p1.axis.visible = False
# Export map to an html file
# output_file("accessibility_map.html")

color_bar = ColorBar(color_mapper=mapper['transform'], width=20,  location=(0,0))

p1.add_layout(color_bar, 'right')
p1.legend.click_policy="hide"

# Export to png for illustration purpose in markdown
export_png(p1, filename="./img/accessibility_parks.png")


In [ ]:
show(p1)

<img src="./img/accessibility_parks.png" width="70%">

#### Visualisation of the parks "*cover*" and so "*holes*" in this cover

In [ ]:
#Make one object with parks and accessibility to get "holes"
from shapely.ops import cascaded_union

all_name = "./data/Lyon_metropole/all.geojson"

list_parks = gpd.GeoDataFrame.from_file(park_name)["geometry"].tolist()
parks = cascaded_union(list_parks)
list_polys = access.union.vis["geometry"].tolist()
access_polys = cascaded_union(list_polys)
polys = cascaded_union([parks, access_polys])
gdf = gpd.GeoDataFrame(geometry=[polys])
gdf.crs = {"init":"epsg:3857"}
gdf.to_file(all_name, driver="GeoJSON")
all_source = geojson_to_datasource(all_name)

# Create Bokeh figures
p2 = figure(
        width = 800,
        height = 600,
        output_backend="webgl"
    )

# Add background tiles
p2.add_tile(get_provider(Vendors.STAMEN_TONER_BACKGROUND), alpha=0.2)

p2.multi_polygons(
    xs="xs",
    ys="ys",
    fill_color="green",
    fill_alpha=0.5,
    line_color="white",
    line_alpha=1.0,
    line_width=1,
    source=all_source,
    legend="Parks_cover"
)

# Export to png for illustration purpose in markdown
export_png(p2, filename="./img/holes_map.png")

show(p2)

<img src="./img/holes_map.png" width="70%">